# Deep stock

## Imports

In [1]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from keras import Sequential
from keras.layers import Input, LSTM, Dropout, LeakyReLU, Dense

## Globals
- epochs
- dataframe
- defined scaler

In [2]:
look_back = 90      # considering number of days to predict the next value
look_ahead = 10     # number of next future values

epochs = 1000
scaler = MinMaxScaler()

df = yf.download( "EURPLN=X", end="2090-01-01" )
start_date = datetime( 2021, 1, 1 )

display_training = False
display_results = False

test_quarter_gains = {}

[*********************100%***********************]  1 of 1 completed


## Functions
- get analyzed dataframe
- model training
- model diagnostics visualization
- adding months
- simulator

In [3]:
#===============================================================
#   PLOT DISPLAY
#===============================================================

def view_data( df ):

    """
    :param df:      dataframe with currency data;
    """

    plt.plot( df )
    plt.show()

    return None



def display_diagnostics( history ):

    """
    :param history:     model training history;
    """

    for epoch in range( 0, epochs, epochs ):
        plt.figure( figsize=( 10, 5 ) )

        plt.subplot( 1, 2, 1 )
        plt.plot( history.history[ 'accuracy' ], label='Training Accuracy' )
        plt.plot( history.history[ 'val_accuracy' ], label='Validation Accuracy' )
        plt.title( 'Training and Validation Accuracy' )
        plt.xlabel( 'Epochs' )
        plt.ylabel( 'Accuracy' )
        plt.legend()

        plt.subplot( 1, 2, 2 )
        plt.plot( history.history[ 'loss' ], label='Training Loss' )
        plt.plot( history.history[ 'val_loss' ], label='Validation Loss' )
        plt.title( 'Training and Validation Loss' )
        plt.xlabel( 'Epochs' )
        plt.ylabel( 'Loss' )
        plt.legend()

        plt.tight_layout()
        plt.grid()
        plt.show()

    return



def display_dataet( dataset ):

    print( f"dataset:\n{ dataset }" )

    return



#===============================================================
#   DATA DISPLAY
#===============================================================

def get_xy_classes( df, look_back : int, view_data=True ):

        """
        :param df:              dataframe,
        :look_back:             number of data considered when making a prediction,
        :param look_ahead:      number of data points to look into the future;

        returns:                X_class, y_class;

        """

        print( '#===============================================================\n \
        #get_xy_classes() note: \
        \n#===============================================================' )


        df = df[ [ 'Close' ] ]
        X_class = []
        y_class = []

        df[ 'Direction' ] = ( df[ 'Close' ].shift(-1) > df[ 'Close' ] ).astype( int )
        df.dropna( inplace=True )   # last column

        for i in range( len( df ) - look_back - 1 ):
            X_class.append( df[ 'Close' ][ i : ( i+look_back )].values )
            y_class.append( df[ 'Direction' ][ i + look_back ] )


        X_class = np.array( X_class )
        y_class = np.array( y_class )
        y_class = y_class.reshape( -1, 1 )

        if ( len( X_class ) == len( y_class ) ): print( "lengths match" )
        else: print( "lengths dont match" )

        if ( view_data ):
                print( f'dataframe:\n{ df }' )
                print( f'X_class:\n{ X_class }\n length:{ len(X_class) }\ny_class:\n{ y_class }\nlength: { len( y_class ) }' )

        print( f'X_class shape: { X_class.shape }\ny_class shape: { y_class.shape }' )

        return  X_class, y_class



#===============================================================
#   MODEL TRAINING
#===============================================================

def train_model( df, scaler, look_back : int, look_ahead: int ):

    """
    :param df:              dataframe with currency data,
    :param scaler:          data scaler,
    :param look_ahead:      amount of window units into the future;
    """

    X_class, y_class = get_xy_classes( df, look_back, view_data=False )

    print( f'X_class:\n{ X_class }' )
    print( f'y_class:\n{ y_class }' )


    X_class_scaled = scaler.fit_transform( X_class )            # X_class
    X_class_reshaped = X_class_scaled.reshape( X_class_scaled.shape[ 0 ] , X_class_scaled.shape[ 1 ] , 1 )
    print( f'X_class_scaled shape: { X_class_scaled.shape } ' )
    print( f'X_class_reshaped shape: { X_class_reshaped.shape }' )


    y_class_scaled = scaler.fit_transform( y_class )            # y_class
    y_class_reshaped = y_class_scaled.reshape( -1, 1, 1 )
    print( f'y_class_scaled shape: { y_class_scaled.shape } ' )
    print( f'y_class_reshaped shape: { y_class_reshaped.shape }' )


    # X_train_class, X_test_class, y_train_class, y_test_class = train_test_split( X_class_reshaped, y_class, test_size=0.2, random_state=42 )


    binary_model = Sequential([
        Input( shape=( -1, 1, 1 ) ),
        LSTM( 128, return_sequences=True ),
        LSTM( 64 ),
        Dense( 64, activation='relu' ),
        Dense( look_ahead, activation='sigmoid' )
    ])


    binary_model.compile( optimizer='adam', loss='binary_crossentropy', metrics=[ 'accuracy' ] )
    history = binary_model.fit( X_train_class, y_train_class, batch_size=64, epochs=epochs, validation_data=( X_test_class, y_test_class ) )

    display_dataet( df )

    return binary_model, history, X_train_class, X_test_class, y_train_class, y_test_class



def get_analyzed_df( df ):

    """
    :param df:      dataframe;
    """

    df = df[ [ 'Close' ] ]
    df[ 'FutureClose' ] = df[ 'Close' ].shift( -5 )
    df[ 'Direction' ] = np.where( df[ 'FutureClose' ] > df[ 'Close' ], 1, 0 )
    df = df.dropna()

    df.index = pd.to_datetime( df.index )

    return df



def add_months( sourcedate, months ):

    """
    :param sourcedate:      main date,
    :param months:          months to add to date;
    """

    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min( sourcedate.day, 28 )

    return datetime( year, month, day )


## Testing

In [4]:
# displaying data
# print( df )
# x_class, y_class = get_xy_classes( df, look_back )

train_model( df, scaler, look_back, look_ahead )

#===============================================================
         #get_xy_classes() note:         
#===============================================================


<ipython-input-3-e70967a3cdf8>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ 'Direction' ] = ( df[ 'Close' ].shift(-1) > df[ 'Close' ] ).astype( int )
<ipython-input-3-e70967a3cdf8>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna( inplace=True )   # last column
<ipython-input-3-e70967a3cdf8>:88: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_class.append( df

lengths match
X_class shape: (5812, 90, 1)
y_class shape: (5812, 1)
X_class:
[[[3.57380009]
  [3.60890007]
  [3.60430002]
  ...
  [4.09310007]
  [4.12739992]
  [4.1322999 ]]

 [[3.60890007]
  [3.60430002]
  [3.59859991]
  ...
  [4.12739992]
  [4.1322999 ]
  [4.09299994]]

 [[3.60430002]
  [3.59859991]
  [3.59290004]
  ...
  [4.1322999 ]
  [4.09299994]
  [4.07359982]]

 ...

 [[4.26776981]
  [4.27259016]
  [4.26802015]
  ...
  [4.25017977]
  [4.24860001]
  [4.22182989]]

 [[4.27259016]
  [4.26802015]
  [4.26893997]
  ...
  [4.24860001]
  [4.22182989]
  [4.2125802 ]]

 [[4.26802015]
  [4.26893997]
  [4.25040007]
  ...
  [4.22182989]
  [4.2125802 ]
  [4.21001005]]]
y_class:
[[0]
 [1]
 [0]
 ...
 [0]
 [1]
 [0]]


ValueError: Found array with dim 3. MinMaxScaler expected <= 2.

In [5]:
binary_model, history, X_train_class, X_test_class, y_train_class, y_test_class = train_model( df, scaler, 5 )
display_diagnostics( history )

TypeError: train_model() missing 1 required positional argument: 'look_ahead'

## Testing gains
- simulating gains (give 1000 and get the model to trade)
- forecast data from 01.01.2021 up until latest date
- forecast data from 01.01.2021 for all quarters of year(s)

### Simulating gains (give 1000 and get the model to trade)

In [ ]:
def get_column_data( df, val_00 : str , val_01 : str ):

        """
        :param df:          dataframe with currency data,
        :param val_00:      first data column,
        :param val_01:      second data column;
        """

        return df[ [ val_00 ] ].values, df[ val_01 ]



def simulator( model, df, initial_balance : float, look_ahead : int ):

    """
    :param model:               trained ML model,
    :param df:                  dataframe with currency data,
    :param initial_balance:     initial balance,
    :param look_ahead:          amount of window units into the future;
    """

    df = get_analyzed_df( df )
    X, y = get_column_data( df, 'Close', 'Direction' )
    future_prices = []

    cash_balance = initial_balance
    foreign_currency_balance = 0

    for i in range( len( X ) - look_ahead ):
        current_data = X[ i : i+1 ]
        future_price_pred = model.predict( current_data.reshape( 1, -1, 1 ) )
        future_prices.append( future_price_pred )


    for i in range( look_ahead, len(X) ):
        current_price = X[ i, 0 ]
        future_price = future_prices[ i - 5 ]
        prediction = y[ i ]

        if ( prediction == 1 ):
            buy_amount = 1.0 * cash_balance
            foreign_currency_balance += buy_amount / current_price
            cash_balance -= buy_amount

        elif ( prediction == 0 ):
            sell_amount = 1.0 * foreign_currency_balance
            cash_balance += sell_amount * current_price
            foreign_currency_balance -= sell_amount

    gains = cash_balance + ( foreign_currency_balance * current_price )
    print( f'gains: \n{ gains }' )

    return gains

In [ ]:
gains = simulator( binary_model, df, 1000, 5 )

<ipython-input-3-419a0be40e5d>:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ 'FutureClose' ] = df[ 'Close' ].shift( -5 )
<ipython-input-3-419a0be40e5d>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ 'Direction' ] = np.where( df[ 'FutureClose' ] > df[ 'Close' ], 1, 0 )


1/1 [==============================] - 0s 22ms/step


### forecast data from 01.01.2021 up until latest date

In [ ]:
def get_analyzed_df( df ):

    """
    :param df:      dataframe to modify;
    """

    df = df[ ['Close'] ]
    df[ 'FutureClose' ] = df[ 'Close' ].shift( -5 )
    df[ 'Direction' ] = np.where( df['FutureClose'] > df['Close'], 1, 0 )
    df = df.dropna()

    df.index = pd.to_datetime( df.index )

    return df



def add_months( sourcedate, months ):

    """
    :param sourcedate:      date to add months to,
    :param months:          months to add;
    """

    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min( sourcedate.day, 28 )

    return datetime( year, month, day )



def test_gains( df, start_date, steps: int, view_results=True ):

    """
    :param start_date:      starting date point (turned into datetime),
    :param end_date:        end date point (datetime object),
    :param steps:           how many months ahead to max date to go,
    :param view_results:    final results;
    """

    ls_final_balances = []
    df = get_analyzed_df( df )
    end_date = df.index.max()

    scaler = MinMaxScaler()

    current_date = start_date

    while ( current_date <= end_date ):

        sub_df = df[ df.index <= current_date ]         # setting a data range
        test_range_df = df[ current_date: ]

        # model training
        binary_model, history, X_train_class, X_test_class, y_train_class, y_test_class = train_model( sub_df, scaler, 5 )

        # run model simulation
        cycle_gain = simulator( binary_model, test_range_df, 10000.0, 5 )
        ls_final_balances.append( cycle_gain )

        # go to new date
        current_date += relativedelta( months=steps )

    if ( view_results ): print( f'All gains: { ls_final_balances }' )

    return None

In [ ]:
test_gains( df, start_date, 3 )

dataframe:
              Close  Direction_1  Direction_2  Direction_3  Direction_4  \
Date                                                                      
2002-04-01  3.57380         True         True         True         True   
2002-04-02  3.60890        False        False        False        False   
2002-04-03  3.60430        False        False        False        False   
2002-04-04  3.59860        False        False        False        False   
2002-04-05  3.59290        False         True         True        False   
...             ...          ...          ...          ...          ...   
2020-12-28  4.49953        False         True         True         True   
2020-12-29  4.48940         True         True         True        False   
2020-12-30  4.53913         True        False        False        False   
2020-12-31  4.57891        False        False        False        False   
2021-01-01  4.53210        False        False        False        False   

            D

<ipython-input-7-45871357b7f4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ 'FutureClose' ] = df[ 'Close' ].shift( -5 )
<ipython-input-7-45871357b7f4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ 'Direction' ] = np.where( df['FutureClose'] > df['Close'], 1, 0 )
<ipython-input-3-419a0be40e5d>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

Epoch 1/1000
61/61 [==============================] - 6s 22ms/step - loss: 0.6931 - accuracy: 0.1232 - val_loss: 0.6927 - val_accuracy: 0.7110
Epoch 2/1000
61/61 [==============================] - 0s 8ms/step - loss: 0.6928 - accuracy: 0.7536 - val_loss: 0.6923 - val_accuracy: 0.7110
Epoch 3/1000
61/61 [==============================] - 0s 8ms/step - loss: 0.6928 - accuracy: 0.7536 - val_loss: 0.6924 - val_accuracy: 0.7110
Epoch 4/1000
61/61 [==============================] - 0s 8ms/step - loss: 0.6928 - accuracy: 0.7536 - val_loss: 0.6923 - val_accuracy: 0.7110
Epoch 5/1000
61/61 [==============================] - 0s 8ms/step - loss: 0.6928 - accuracy: 0.7536 - val_loss: 0.6924 - val_accuracy: 0.7110
Epoch 6/1000
61/61 [==============================] - 0s 8ms/step - loss: 0.6928 - accuracy: 0.7536 - val_loss: 0.6924 - val_accuracy: 0.7110
Epoch 7/1000
61/61 [==============================] - 1s 9ms/step - loss: 0.6928 - accuracy: 0.7536 - val_loss: 0.6924 - val_accuracy: 0.7110
Epoch

KeyboardInterrupt: 

### forecast data from 01.01.2021 for all quarters of year(s)

In [ ]:
def test_gains_quarter( df, steps: int, view_results=True ):

    """
    :param df:              dataframe with other data,
    :param steps:           how many months ahead to max date to go,
    :param view_results:    final results;
    """

    forecast_win = 10
    ls_final_balances = []

    start_date = df.index.min()
    end_date = df.index.max()
    current_date = start_date

    while ( current_date <= end_date ):
        interval_end_date = current_date + relativedelta( months=steps )

        sub_df = df[ ( df.index >= current_date ) & ( df.index <= interval_end_date ) ]

        forecast_date = current_date

        while ( forecast_date <= interval_end_date - timedelta( days=forecast_win ) ):
            test_range_df = df[ forecast_date: forecast_date + timedelta( days=forecast_win ) ]

            binary_model, history, X_train_class, X_test_class, y_train_class, y_test_class = train_model( test_range_df, scaler, 5 )

            cycle_gain = simulator( binary_model, test_range_df, 1000, 5 )
            ls_final_balances.append( cycle_gain )

            forecast_date += timedelta( days=1 )

        current_date += relativedelta( months=steps )

    return ls_final_balances

In [ ]:
ls_final_balances = test_gains_quarter( df, 3 )

## Even more testing